<a href="https://colab.research.google.com/github/Shiye-Chen/minDE/blob/main/ROI_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from skimage import io

# 读取 Cellpose mask 16-bit TIFF
cellpose_mask = io.imread("/content/ROI/MASK_0000_8bit.tif")

# 读取 Hyperstack TIFF（假设是三通道）
hyperstack = io.imread("/content/ROI/0000_merged.tiff")

print(f"Cellpose mask 维度: {cellpose_mask.shape}, 数据类型: {cellpose_mask.dtype}")
print(f"Hyperstack 维度: {hyperstack.shape}, 数据类型: {hyperstack.dtype}")


Cellpose mask 维度: (1184, 2216), 数据类型: uint16
Hyperstack 维度: (1184, 2216, 3), 数据类型: uint8


In [2]:
from skimage import img_as_ubyte
import numpy as np

# 将 16-bit Cellpose mask 转换为 8-bit
cellpose_mask_8bit = img_as_ubyte(cellpose_mask / np.max(cellpose_mask))

print(f"转换后 Cellpose mask 数据类型: {cellpose_mask_8bit.dtype}")

转换后 Cellpose mask 数据类型: uint8


In [3]:
# 确保两幅图像尺寸完全匹配
assert cellpose_mask_8bit.shape == hyperstack.shape[:2], "图像尺寸不一致！"


In [5]:
!pip install roifile
import roifile

# 读取 Hyperstack TIFF 图像
hyperstack = io.imread("/content/ROI/0000_merged.tiff")

# 分离 RGB 通道
red_channel = hyperstack[:, :, 0]
green_channel = hyperstack[:, :, 1]
blue_channel = hyperstack[:, :, 2]

# 读取 ImageJ 导出的 ROI 文件
roi_path = "/content/ROI/0000_RoiSet.zip"
rois = roifile.ImagejRoi.fromfile(roi_path)

print(f"成功读取 {len(rois)} 个 ROI。")


成功读取 1145 个 ROI。


In [12]:
# 读取 ROI 文件
roi_path = "/content/ROI/0000_RoiSet.zip"
rois = roifile.ImagejRoi.fromfile(roi_path)

# 创建字典，将 ROI 按细胞归类
cell_roi_dict = {}

# 遍历每个 ROI
for idx, roi in enumerate(rois):
    roi_name = roi.name if roi.name else f"ROI_{idx + 1}"

    # 根据 ROI 名称提取细胞 ID
    cell_id = roi_name.split("_")[0]  # 假设名称类似 "Cell1_ROI1"

    # 将 ROI 归类到对应的细胞
    if cell_id not in cell_roi_dict:
        cell_roi_dict[cell_id] = []

    cell_roi_dict[cell_id].append(idx + 1)  # 保存 ROI ID

# 查看归类结果
for cell, roi_ids in list(cell_roi_dict.items())[:5]:
    print(f"{cell}: {roi_ids}")


0011-0279: [1]
0003-0300: [2]
0004-0328: [3]
0003-0351: [4]
0008-0431: [5]


In [7]:
# 设置信号强度阈值
signal_threshold = 20

# 存储分析结果
results = []

# 遍历每个 ROI
for idx, roi in enumerate(rois):
    # 获取 ROI 区域坐标
    coords = roi.coordinates()
    coords = np.round(coords).astype(int)  # 确保是整数坐标

    # 提取该 ROI 内的信号
    red_signal = []
    green_signal = []
    blue_signal = []

    for y, x in coords:
        if 0 <= y < hyperstack.shape[0] and 0 <= x < hyperstack.shape[1]:
            red_signal.append(red_channel[y, x])
            green_signal.append(green_channel[y, x])
            blue_signal.append(blue_channel[y, x])

    # 计算信号强度
    red_mean = np.mean(red_signal) if red_signal else 0
    green_mean = np.mean(green_signal) if green_signal else 0
    blue_mean = np.mean(blue_signal) if blue_signal else 0

    # 判断信号存在
    red_present = any(v > signal_threshold for v in red_signal)
    green_present = any(v > signal_threshold for v in green_signal)
    blue_present = any(v > signal_threshold for v in blue_signal)

    # 保存结果
    results.append({
        "ROI_ID": idx + 1,
        "Red_Mean_Intensity": red_mean,
        "Green_Mean_Intensity": green_mean,
        "Blue_Mean_Intensity": blue_mean,
        "Red_Signal_Present": red_present,
        "Green_Signal_Present": green_present,
        "Blue_Signal_Present": blue_present
    })

print(f"✅ 成功分析 {len(results)} 个 ROI。")


✅ 成功分析 1145 个 ROI。


In [10]:
import pandas as pd

# 转换为 DataFrame
df = pd.DataFrame(results)

# 保存为 CSV
output_csv_path = "/content/ROI/0000_Analysis_1.csv"
df.to_csv(output_csv_path, index=False)

print(f"📄 结果已保存到：{output_csv_path}")

# 显示前几行结果
df.head()


📄 结果已保存到：/content/ROI/0000_Analysis_1.csv


,ROI_ID,Red_Mean_Intensity,Green_Mean_Intensity,Blue_Mean_Intensity,Red_Signal_Present,Green_Signal_Present,Blue_Signal_Present
0,1,19.627451,76.000000,75.784314,True,True,True
1,2,4.600000,58.160000,76.880000,False,True,True
2,3,3.935484,33.935484,78.129032,False,True,True
3,4,3.040000,21.880000,72.080000,False,True,True
4,5,30.457143,44.000000,89.085714,True,True,True
